In [1]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
from teamName import getMyPosition 

nInst = 0
nt = 0
commRate = 0.0010
dlrPosLimit = 10000


def loadPrices(fn):
    global nt, nInst
    df = pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    (nt, nInst) = df.shape
    return (df.values).T


pricesFile = "./prices.txt"
prcAll = loadPrices(pricesFile)
print("Loaded %d instruments for %d days" % (nInst, nt))




Loaded 50 instruments for 500 days


#### EMA Calculating Function

In [3]:
# Function to calculate EMA
def calculate_ema(prices, period):
    alpha = 2 / (period + 1)
    ema = np.zeros_like(prices)
    ema[0] = prices[0]  # Initialize with the first price
    for t in range(1, len(prices)):
        ema[t] = alpha * prices[t] + (1 - alpha) * ema[t-1]
    return ema

#### MACD Trading Strategy installed into getMyPosition Function

#### Optimise the days of shorter and longer EMA so that trading signals

### Check my code inside getMyPosition whether is wriiten correctly or not

In [5]:
##### TODO #########################################
### RENAME THIS FILE TO YOUR TEAM NAME #############
### IMPLEMENT 'getMyPosition' FUNCTION #############
### TO RUN, RUN 'eval.py' ##########################

nInst = 50
currentPos = np.zeros(nInst)

def getMyPosition(prcSoFar):
    global currentPos
    (nins, nt) = prcSoFar.shape
    if nt < 26:
        return np.zeros(nins)  # Not enough data points to calculate EMAs
    
    # Initialize positions
    new_positions = np.zeros(nins)
    # Iterate over each instrument
    for i in range(nins):
        prices = prcSoFar[i, :]

        # Calculate 12-period and 26-period EMAs
        ema_12 = calculate_ema(prices, 12)   ### optimise the day here
        ema_26 = calculate_ema(prices, 26)   ### optimise the day here

        # Current price of the stock
        current_price = prices[-1]
        
        # Generate trading signals based on EMA crossover
        if ema_12[-1] > ema_26[-1] and ema_12[-2] <= ema_26[-2]:
            # Buy signal: 12-period EMA crosses above 26-period EMA
            # Calculate the number of shares to buy without exceeding $10,000
            num_shares_to_buy = 10000 // current_price
            new_positions[i] = num_shares_to_buy
        elif ema_12[-1] < ema_26[-1] and ema_12[-2] >= ema_26[-2]:
            # Sell signal: 12-period EMA crosses below 26-period EMA
            # Calculate the number of shares to sell (short) without exceeding $10,000
            num_shares_to_sell = -10000 // current_price
            new_positions[i] = num_shares_to_sell
        else:
            # No signal, maintain current position
            new_positions[i] = currentPos[i] 
    # Update current positions
    
    currentPos = new_positions
    return currentPos


#### P&L Function  optimise the trading at line for t in range(50,501)

In [7]:
def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_, nt) = prcHist.shape
    for t in range(50, 501):   ### day start to trade at 50 to 500
        prcHistSoFar = prcHist[:, :t]
        newPosOrig = getMyPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:, -1]
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        newPos = np.clip(newPosOrig, -posLimits, posLimits)
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
        ### this is my modification code for checking position
        #print(newPosOrig) ###check position Number of holding shares
        #print(curPrices)
        #print(newPos)   ### position number of holding shares after apply limited constraints
        print("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" %
              (t, value, todayPL, totDVolume, ret))
    pll = np.array(todayPLL)
    (plmu, plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(250) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)

#### Calculating P&L by calling calcPL function with stock price input

In [9]:
(meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
score = meanpl - 0.1*plstd
print("=====")
print("mean(PL): %.1lf" % meanpl)
print("return: %.5lf" % ret)
print("StdDev(PL): %.2lf" % plstd)
print("annSharpe(PL): %.2lf " % sharpe)
print("totDvolume: %.0lf " % dvol)
print("Score: %.2lf" % score)

Day 50 value: -10.00 todayPL: $-10.00 $-traded: 9997 return: -0.00100
Day 51 value: 82.38 todayPL: $92.37 $-traded: 40023 return: 0.00206
Day 52 value: 330.87 todayPL: $248.50 $-traded: 50057 return: 0.00661
Day 53 value: 270.19 todayPL: $-60.69 $-traded: 70222 return: 0.00385
Day 54 value: 678.11 todayPL: $407.92 $-traded: 70480 return: 0.00962
Day 55 value: 548.31 todayPL: $-129.80 $-traded: 70747 return: 0.00775
Day 56 value: 476.43 todayPL: $-71.88 $-traded: 90809 return: 0.00525
Day 57 value: 875.58 todayPL: $399.15 $-traded: 130779 return: 0.00670
Day 58 value: 628.61 todayPL: $-246.97 $-traded: 160954 return: 0.00391
Day 59 value: 200.66 todayPL: $-427.94 $-traded: 190947 return: 0.00105
Day 60 value: -236.20 todayPL: $-436.87 $-traded: 240974 return: -0.00098
Day 61 value: -255.92 todayPL: $-19.71 $-traded: 261029 return: -0.00098
Day 62 value: -203.57 todayPL: $52.35 $-traded: 300993 return: -0.00068
Day 63 value: -246.34 todayPL: $-42.77 $-traded: 351350 return: -0.00070
Day 